In [203]:
import json
import pandas as pd

json_file = '../01_scraping/json/espn_2022calendar.json'
df = pd.read_json(json_file)

#df_wk1 = df.loc[(df.week==1), ["awayteam","hometeam"]]
#df_wk1

df = df.replace("Commanders", "Washington")

#df_wk1.to_csv('../04_datasets/nfl2022_week1.csv')
df.to_csv('../04_datasets/nfl2022_total.csv')

In [204]:
json_file = '../01_scraping/json/odds_2021.json'
df_odds = pd.read_json(json_file)
#df.drop_duplicates(keep = 'first', inplace=True)
df_odds.drop_duplicates(keep='first', inplace=True)
#df_odds

df_teams = pd.read_csv('../05_working_files/team_correspondance_list.csv',header=None,sep=";")
df_teams.columns = ['abbr', 'team']
df_teams

df_2021_preds = pd.read_csv('../07_predict/prediction_xgb_games_2021.csv')
df_2021_preds
df_2021_preds = df_2021_preds[["week", "awayteam", "hometeam","pred_winner", "winner_home"]]

#df_2021_preds = df_2021_preds.assign(Winner = lambda x: (x['Total_Marks'] /500 * 100))
df_2021_preds = df_2021_preds.assign(pred_winner_team='nobody')
df_2021_preds = df_2021_preds.assign(real_winner='nobody')
df_2021_preds.pred_winner_team = df_2021_preds[['awayteam','hometeam','pred_winner']].apply(lambda x: x['awayteam'] if x['pred_winner']==0 else x['hometeam'], axis=1)
df_2021_preds.real_winner = df_2021_preds[['awayteam','hometeam','winner_home']].apply(lambda x: x['awayteam'] if x['winner_home']==0 else x['hometeam'], axis=1)
df_2021_preds.sort_values(by=['week','awayteam'],inplace=True)
df_2021_preds = df_2021_preds[["week", "awayteam", "hometeam","pred_winner_team", "real_winner"]]
#df_2021_preds.drop('pred_winner')

df_final = df_2021_preds
df_final

,week,awayteam,hometeam,pred_winner_team,real_winner
1,1,49ers,Lions,49ers,49ers
214,1,Bears,Rams,Rams,Rams
213,1,Broncos,Giants,Broncos,Broncos
209,1,Browns,Chiefs,Browns,Chiefs
8,1,Cardinals,Titans,Cardinals,Cardinals
...,...,...,...,...,...
9,18,Saints,Falcons,Saints,Saints
3,18,Seahawks,Cardinals,Seahawks,Seahawks
19,18,Steelers,Ravens,Steelers,Steelers
11,18,Titans,Texans,Texans,Titans


In [205]:
#from calendar import week


df_scores = pd.read_json('../01_scraping/json/espn_scores.json')
#df.drop_duplicates(keep = 'first', inplace=True)
df_scores = df_scores.loc[(df_scores.season==2021)]


#df_2021_preds = df_2021_preds[["week", "awayteam", "hometeam","pred_winner"]]

df_scores=  df_scores[["week","awayteam","hometeam","awayscore","homescore"]]
df_scores = df_scores.assign(real_winner='nobody')
df_scores = df_scores.assign(scoreabs=df_scores.awayscore-df_scores.homescore)

#df_scores = df_scores.assign(predicted_winner='nobody')
#df_scores.predicted_winner = df_2021_preds[['week','hometeam','awayteam',]].apply(lambda x: x['awayteam'] if x['scoreabs']>0 else x['hometeam'], axis=1)
df_scores.real_winner = df_scores[['awayteam','hometeam','scoreabs']].apply(lambda x: x['awayteam'] if x['scoreabs']>0 else x['hometeam'], axis=1)
df_scores=  df_scores[["week","awayteam","hometeam","real_winner"]]
df_scores

#df_2021_preds.update(df_scores)
df_2021_preds = df_2021_preds[['week','awayteam','hometeam','pred_winner_team']]

#df_2021_preds = df_2021_preds.rename(columns={'pred_winner_team': 'predicter_winner'},axis=1)

df_scores.week.astype('int')
df_2021_preds.week.astype('int')

#df_2021_preds.loc[:, ['week','awayteam','hometeam']] = df_scores[['week','awayteam','hometeam']]
df_2021_preds = df_2021_preds.reset_index(drop=True)
df_2021_preds


,week,awayteam,hometeam,pred_winner_team
0,1,49ers,Lions,49ers
1,1,Bears,Rams,Rams
2,1,Broncos,Giants,Broncos
3,1,Browns,Chiefs,Browns
4,1,Cardinals,Titans,Cardinals
...,...,...,...,...
266,18,Saints,Falcons,Saints
267,18,Seahawks,Cardinals,Seahawks
268,18,Steelers,Ravens,Steelers
269,18,Titans,Texans,Texans


In [206]:
df_scores.sort_values(by=['week','awayteam'],inplace=True)
df_scores = df_scores.reset_index(drop=True)
df_scores

,week,awayteam,hometeam,real_winner
0,1,49ers,Lions,49ers
1,1,Bears,Rams,Rams
2,1,Broncos,Giants,Broncos
3,1,Browns,Chiefs,Chiefs
4,1,Cardinals,Titans,Cardinals
...,...,...,...,...
267,18,Saints,Falcons,Saints
268,18,Seahawks,Cardinals,Seahawks
269,18,Steelers,Ravens,Steelers
270,18,Titans,Texans,Titans


In [207]:
df_final
df_odds
df_teams

#df_scores = df_scores.assign(real_winner='nobody')
df_final = df_final.reset_index(drop=True)
#df_final = df_final.assign(team_abbr='SF')
#df_final = df_final.assign(team=df_final.real_winner)
df_final = df_final.assign(team=df_final.pred_winner_team)
df_final

df_pouet = pd.merge(df_final,df_teams, on='team', how='left')

df_pouet['team'] = df_pouet['abbr']
#df.loc[df['line_race'] == 0, 'rating'] = 0
#df_pouet.loc[df_pouet['abbr'] == 'LA' , df_pouet.abbr= 'LAR']
#df['rating'] = np.where(df['line_race']==0, 0, df['rating'])
#LA = df_pouet.loc[df_pouet.team=='LA']

#df_pouet.loc[df.team == 'LA', 'team'] = 'LAR'
df_pouet = df_pouet.replace('LA','LAR')
bets = pd.merge(df_pouet,df_odds, on=['week','team'], how='left')
#bets = bets.loc[(bets.moneyline=='-100')] 
bets = bets.assign(money_bet=100)
bets = bets.assign(good_predict='Yes')
bets.good_predict = bets[['pred_winner_team','real_winner']].apply(lambda x: 'Yes' if x['pred_winner_team']==x['real_winner'] else 'No', axis=1)
bets = bets.assign(pocket=0)
#bets = bets.money_bet.astype('int64')
#bets = bets.moneyline.astype('int64')
#df.Weight.astype('int64')
#bets.pocket = bets[['good_predict','money_bet','moneyline']].apply(lambda x: (x['moneybet']*100/x['moneyline'])+100 if (x['moneyline']<<0 & x['good_predict']=='Yes') else 0, axis=1)
#bets.dtypes
#bets['moneyline'].isnull().values.any()
bets


,week,awayteam,hometeam,pred_winner_team,real_winner,team,abbr,moneyline,money_bet,good_predict,pocket
0,1,49ers,Lions,49ers,49ers,SF,SF,-385,100,Yes,0
1,1,Bears,Rams,Rams,Rams,LAR,LAR,-360,100,Yes,0
2,1,Broncos,Giants,Broncos,Broncos,DEN,DEN,-155,100,Yes,0
3,1,Browns,Chiefs,Browns,Chiefs,CLE,CLE,210,100,No,0
4,1,Cardinals,Titans,Cardinals,Cardinals,ARI,ARI,135,100,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...
266,18,Saints,Falcons,Saints,Saints,NO,NO,-185,100,Yes,0
267,18,Seahawks,Cardinals,Seahawks,Seahawks,SEA,SEA,210,100,Yes,0
268,18,Steelers,Ravens,Steelers,Steelers,PIT,PIT,165,100,Yes,0
269,18,Titans,Texans,Texans,Titans,HOU,HOU,410,100,No,0


In [218]:
'''
def f(x):
#x['streak'] = x.groupby( (x['loser']!=x['team']).cumsum()).cumcount()+( (x['team'] != x['loser']).cumsum() == 0).astype(int) 
    if (x['good_predict']=='Yes' & x['moneyline']<0):
        x['pocket']=(x['money_bet']*100/abs(x['moneyline']))+100
    elif(x['good_predict']=='Yes' & x['moneyline']>0):
        x['pocket']=(x['money_bet']+(x['moneyline']))   
    else:
        x['pocket']=-100
    return x
'''
bets = bets.moneyline.astype('str')
bets.dtypes
#bets.loc[(bets['moneyline']<0 & bets['good_predict']=='Yes'), ['pocket']] = (bets['money_bet']*100/abs(bets['moneyline']))+100

#bets

week                 int64
awayteam            object
hometeam            object
pred_winner_team    object
real_winner         object
team                object
abbr                object
moneyline            int64
money_bet            int64
good_predict        object
pocket               int64
dtype: object